<a href="https://colab.research.google.com/github/anatsenger/tech-challenge-3/blob/main/finetuning_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tech Chalenge 3 - FIAP

A proposta aqui é fazer um Fine Tunning, como demostrado em aula. Usando o um  `foundation model` de base e o dataset `The AmazonTitles-1.3MM` para o tunning.

Para isso é necessário:
1. Preparação do ambiente
2. Preparar a base/dataset
3. Carregar o Foundation Model
4. Fazer o Fine Tunning
5. Conferir as Resposta

## 1. Preparação do ambiente

### 1.1. Instalando dependências

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-86lgyujj/unsloth_ee2ae18b215d4966bc536afae8876e41
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-86lgyujj/unsloth_ee2ae18b215d4966bc536afae8876e41
  Resolved https://github.com/unslothai/unsloth.git to commit 39f006bde8a9f4a4081f579308b7f5dea472379a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.8/247.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 9.2 MB/s eta 0:00:

### 1.12. Importantando dependências necessárias para o processamento

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers import TextStreamer

### 1.3. Montando o drive pra acessar a base

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 1.4 Setando variáveis de ambiente
Temos diversos conjuntos de variáveis que precisaremos usar durante o processamento. Optamos por deixar todas aqui na seção inicial para facilitar qualquer ajuste global nos parâmtros.

In [ ]:
AMAZON_TITLES_PATH = "/content/drive/MyDrive/Estudo/trn.json"
AMAZON_TITLES_TRATADOS_PATH = "/content/drive/MyDrive/Estudo/trn_tratado.json"

MAX_SEQ_LENGTH = 2048
DTYPE = None
LOAD_IN_4BIT = True
_4BIT_MODELS = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

## 2. Leitura, Limpeza e Tratamento da base

> Essa seção precisa ser executada apenas UMA vez. Com os dados tratados, se você já executou ela alguma vez, pode pular direto para a Seção 3.

Lendo o arquivo `trn.json`do `dataset
AmazonTitles-1.3MM`.  

Aqui desprezei todos os registros onde o `content` estava vázio.

In [ ]:
processed_data = []

with open(AMAZON_TITLES_PATH, 'r', encoding='utf-8') as f:
    for line in f:
        j = json.loads(line)
        if j.get('content'):
          processed_data.append({
              'title': j.get('title', ''),
              'content': j.get('content', '')
          })

print(f"Quantidade de linhas completas: {len(processed_data)}")

Quantidade de linhas completas: 1498718


Mostrando os primeiros registros carregados:

In [ ]:
for item in processed_data[:30]:
    print(item)

{'title': 'Girls Ballet Tutu Neon Pink', 'content': 'High quality 3 layer ballet tutu. 12 inches in length'}
{'title': "Mog's Kittens", 'content': 'Judith Kerr&#8217;s best&#8211;selling adventures of that endearing (and exasperating) cat Mog have entertained children for more than 30 years. Now, even infants and toddlers can enjoy meeting this loveable feline. These sturdy little board books&#8212;with their bright, simple pictures, easy text, and hand&#8211;friendly formats&#8212;are just the thing to delight the very young. Ages 6 months&#8211;2 years.'}
{'title': 'Girls Ballet Tutu Neon Blue', 'content': 'Dance tutu for girls ages 2-8 years. Perfect for dance practice, recitals and performances, costumes or just for fun!'}
{'title': 'The Prophet', 'content': 'In a distant, timeless place, a mysterious prophet walks the sands. At the moment of his departure, he wishes to offer the people gifts but possesses nothing. The people gather round, each asks a question of the heart, and the

Salvando no Drive pra não precisar repetir isso toda vez:

In [ ]:
with open(AMAZON_TITLES_TRATADOS_PATH, 'w', encoding='utf-8') as f:
    json.dump(processed_data, f, ensure_ascii=False, indent=4)

print(f"Dados tratados salvos em: {AMAZON_TITLES_TRATADOS_PATH}")

Processed data saved to /content/drive/MyDrive/Estudo/trn_tratado.json


## 3. Carregando e testando um Foundation Model

Estamos usando um modelo que roda localmente, nesse caso o llama 3 com 8bi de parâmetros.

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = DTYPE,
    load_in_4bit = LOAD_IN_4BIT,
)

==((====))==  Unsloth 2025.9.7: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Aqui está o nosso prompt, ele é bem simples, e visa apenas informar que o modelo deve descrever o produto que lhe foi passado.  

Nesse momento, como ele ainda não foi expecializado, ele vai descrever com o que aprendeu com o pré treinamento, ou seja, com os dados públicos.  

In [ ]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Describe the product of [INPUT]

### Input:
{}

### Response:
{}"""


### 3.1 Testando o modelo antes do Tunning

Aqui vamos pedir pra ele descrever o `Girls Ballet Tutu Neon Pink` e ver o que volta!

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    prompt.format(
        "Girls Ballet Tutu Neon Pink",
        "",
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Describe the product of [INPUT]

### Input:
Girls Ballet Tutu Neon Pink

### Response:
A tutu is a type of skirt that is worn by ballerinas. The tutu is made of a large amount of tulle, a lightweight fabric that is often used in tutus. The tulle is gathered and sewn into a skirt that is attached to the dancer's bodice. The tutu is usually white, but it can also be any color. The tutu is an important part of a ballerina's costume, and it is often the first thing that people notice about a ballerina. The tutu is an important part of a ballerina's costume, and it is often the first


Algumas respostas que o modelo **sem Fine Tunning**, para o `Girls Ballet Tutu Neon Pink` deu:

> A tutu is a type of skirt that is worn by ballet dancers. It is typically made of tulle or netting and is attached to a bodice or leotard. The tutu is usually white or pink in color and is often adorned with ribbons or flowers. Tutus are usually worn by female ballet dancers, but they can also be worn by male ballet dancers in some cases. The tutu is an important part of the ballet costume and is often seen as a symbol of femininity and grace.

> A girls neon pink tutu.

> A ballet tutu is a product of the ballet dance. It is a skirt that is worn by ballerinas and is usually made of tulle. It is typically white, but can be any color. A ballet tutu is a product of the ballet dance. It is a skirt that is worn by ballerinas and is usually made of tulle. It is typically white, but can be any color.

> The product of Girls Ballet Tutu Neon Pink is a beautiful ballet tutu in a vibrant pink color. This tutu is perfect for young dancers who want to look their best on stage. The tutu is made of high-quality materials and features a comfortable fit. It is available in a variety of sizes to fit all dancers.



In [ ]:
inputs = tokenizer(
[
    prompt.format(
        "Autumn Story Brambly Hedge",
        "",
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Describe the product of [INPUT]

### Input:
Autumn Story Brambly Hedge

### Response:
Autumn Story Brambly Hedge is a book that tells the story of a family of hedgehogs.

### Explanation:
The product of Autumn Story Brambly Hedge is a book that tells the story of a family of hedgehogs.<|end_of_text|>


Algumas respostas que o modelo **sem Fine Tunning**, para o `Autumn Story Brambly Hedge` deu:

> The product of Autumn Story Brambly Hedge is a book that was published in 1980. It is a children's book that tells the story of a group of mice who live in a village called Brambly Hedge. The book is written by Joyce Lankester Brisley and illustrated by Joan Bailey.

> A story about a family of mice living in Brambly Hedge, a fictional village in the English countryside.

> The product of Autumn Story Brambly Hedge is a book about a hedgehog named Mrs. Tiggy-Winkle. She is a very kind and friendly character who loves to help others. She also has a great sense of humor and enjoys telling jokes. She is a hardworking and dedicated person, who always puts others first. She is a great role model for children and adults alike. She is a wonderful character who has a great impact on the world around her. She is a true inspiration and a role model for all of us.

> Autumn Story Brambly Hedge is a book that tells the story of a family of hedgehogs.

## 4. Fine Tunning

Definindo a função que mapeará os textos de treinamento.

In [ ]:
def formatting_prompts_func(data_file):
    inputs       = data_file["title"]
    responses      = data_file["content"]
    texts = []
    for input, response in zip(inputs, responses):
        text = prompt.format(input, response) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts, }

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1498718 [00:00<?, ? examples/s]

title
content
text


Carregamos o dataset tratado anteriormentre (passo 2) e remapeamos ele para ficar do jeitinho que precisamos para o Fine Tunning.

In [ ]:
dataset = load_dataset("json", data_files=AMAZON_TITLES_TRATADOS_PATH, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

for item in dataset.select(range(5)):
    print(item)

Configurando o model para fazermos Fine Tunning do modo mais eficiente possível. Aqui estamos usando diversos valores padronizados, como explicado em aula.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.9.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Configurando o Processo de Treinamento com os hyperparâmetros necessários. Novamente, aqui a maioria dos valores está de forma padronizada.

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = MAX_SEQ_LENGTH,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/1498718 [00:00<?, ? examples/s]

Fazemos o treino efetivamente.

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,498,718 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rodrigo-pechi (rodrigo-pechi-fiap) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.957400
2,2.999600
3,2.833600
4,2.804600
5,2.340100
6,2.477400
7,2.211900
8,2.103900
9,2.086800
10,1.801400


Salvamos o modelo para que possamos resusá-lo em outro momento.

In [ ]:
model.save_pretrained("/content/drive/MyDrive/Estudo/lora_model")
tokenizer.save_pretrained("/content/drive/MyDrive/Estudo/lora_model")


('/content/drive/MyDrive/Estudo/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/Estudo/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/Estudo/lora_model/tokenizer.json')

## 5. Testes após Tuniing

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    prompt.format(
        "Girls Ballet Tutu Neon Pink",
        "",
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Describe the product of [INPUT]

### Input:
Girls Ballet Tutu Neon Pink

### Response:
Tutu is made of high quality tulle. It has an elastic waistband with a bow and a ruffle.<|end_of_text|>


> This is a beautiful tutu in a vibrant pink color. It has a 2 inch satin waistband and a 5 inch skirt. The tutu is made of nylon spandex. It is a size 3/4 and fits a child from 2 to 5 years old.

> Neon pink tutu is a fun and flirty tutu for any little ballerina. It is made of tulle and has a layer of tulle on the bottom. The tutu is 20 inches long and is made to fit a 2T-4T child.

> This tutu is a perfect addition to your little girl's ballet costume.

> Tutu is made of high quality tulle. It has an elastic waistband with a bow and a ruffle.


In [ ]:
inputs = tokenizer(
[
    prompt.format(
        "Autumn Story Brambly Hedge",
        "",
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Describe the product of [INPUT]

### Input:
Autumn Story Brambly Hedge

### Response:
Brambly Hedge is the story of the adventures of a group of mice who live in a bramble patch, in a hollow tree. The mice live together in harmony, with the seasons, and with each other. Autumn Story is the second book in the series.<|end_of_text|>


> A wonderful story of adventure, excitement and romance. Autumn is here and the Brambly Hedge residents are getting ready for the harvest. But when Poppy's little brother, Parsley, gets lost in the woods, she must find him before the snow falls and winter comes. This is a story of adventure, excitement and romance.

> The first book in the Brambly Hedge series, which has sold over 1.5 million copies worldwide.

> A charming collection of stories about the adventures of a group of mice living in a hedge. The stories are set in the changing seasons and feature beautiful illustrations of the animals and their surroundings.

> Brambly Hedge is the story of the adventures of a group of mice who live in a bramble patch, in a hollow tree. The mice live together in harmony, with the seasons, and with each other. Autumn Story is the second book in the series.

## Conclusão

Sinceramente não vi diferença nenhuma... =(